In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import matplotlib.pyplot as plt
import spacy
import string
import pickle
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [ ]:
df.head()

In [ ]:
#ONE REVIEW
df['review'][0]

# Text Cleaning
>>1. Sample 10000 rows 
>>2. Remove HTML tags
>>3. Remove special characters
>>4. Removing stop words
>>5. Stemming

In [ ]:
#df=df.sample(50000)

In [ ]:
df.shape

It means 50000 rows and 2 columns

In [ ]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

Replacing positive with 1 and negative with 0

In [ ]:
df.head()

*To remove HTML tags*

In [ ]:
import re
clean=re.compile('<.*?>')
re.sub(clean, '', df.iloc[2].review)

In [ ]:
#Function to clean HTML tags
def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
df['review']=df['review'].apply(clean_html)

Now the review column doesn't have any tags.

In [ ]:
#Converting to lowercase

def convert_lower(text):
    return text.lower()

Applying the function to our review column

In [ ]:
df['review']=df['review'].apply(convert_lower)

In [ ]:
#function to remove special characters

def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x + ' '
    return x
    



The string is iterating character by character and for every char I'm asking it is alphanumeric or not.If not then I'll simply replace that thing with a space otherwise I'll just concatenate it into an empty string.

In [ ]:
remove_special('He lives @ Avenue 45')

Applying the remove_special function on our review column.

In [ ]:
df['review']=df['review'].apply(remove_special)

In [ ]:
#Remove stop words

import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

In [ ]:
len(stopwords.words('english'))

I'm creating a stopwords function; in this we are taking a text i.e. each review and hen splitting it to convert into list.If any word of our review is not there in the stopwords list then we will append it into an empty list.
I'm putting all the contents of x into y and then clearing x so that if we get next review that list is empty.

In [ ]:
def remove_stopwords(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y

Applying remove_stopwords function to our review column.


In [ ]:
df['review']=df['review'].apply(remove_stopwords)

In [ ]:
df

In [ ]:
#Now we will perform stemming 

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

We are taking every word of the review and passing it to the stem function.

In [ ]:
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [ ]:
stem_words(['I','loving','loved','loves'])

Applying stem_words function to our review column.

In [ ]:
df['review']=df['review'].apply(stem_words)

In [ ]:
df

In [ ]:
#Join Back

def join_back(list_input):
    return " ".join(list_input)

In [ ]:
df['review']=df['review'].apply(join_back)

In [ ]:
df['review']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1000)

In [ ]:
X=cv.fit_transform(df['review']).toarray()

In [ ]:
X.shape

This means in my data set I have 36526 unique words.

In [ ]:
X[0]

In [ ]:
X[0].max()

This means there is one word in the first review which occurs most of the times i.e. 3 in the first review.

In [ ]:
y=df.iloc[:,-1].values

Extracting the last column which is the labels.

In [ ]:
y

In [ ]:
y.shape

X,y; Splitting my dataset into two parts : Training set and Test set(already known).

We will compare the true value of the test test with the predicted value of the algorithm to get the accuracy.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

Here I have taken the size of the test set to be 30% of the total dataset.

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Finding acuuracy using NAIVE BAYES Model.

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [ ]:
clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()

In [ ]:
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)

In [ ]:
y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf2.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Gaussian",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernoulli",accuracy_score(y_test,y_pred3))

      

In [ ]:
Finding accuracy using SUPPORT VECTOR MACHINE Model.

In [ ]:
from sklearn.svm import SVC
svc_model=SVC()


svc_model.fit(X_train,y_train)
prediction=svc_model.predict(X_test)
print(accuracy_score(y_test,prediction))
print(metrics.confusion_matrix(y_text,prediction))